<a href="https://colab.research.google.com/github/rubensbraz/automatizaUnB-v1-2019/blob/master/v1_Extrac%CC%A7a%CC%83o_MW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Código de Extração das informações do MatrículaWeb UnB - Oferta***




In [0]:
# Instalar Bibliotecas
!pip install pymysql
!pip install html.parser
!pip install html5lib
!pip install lxml
!pip install tqdm


# Importar Bibliotecas
import pymysql.cursors
from datetime import datetime
import urllib.request
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import re


In [0]:
# Modelo de Links das páginas do MatrículaWeb GRADUAÇÃO: https://matriculaweb.unb.br/graduacao/sec/qr.aspx
link_oferta_campus_gr = "https://matriculaweb.unb.br/graduacao/oferta_dep.aspx?cod="
link_oferta_departamento_gr = "https://matriculaweb.unb.br/graduacao/oferta_dis.aspx?cod="
link_oferta_materia_gr = "https://matriculaweb.unb.br/graduacao/oferta_dados.aspx?cod="

# Modelo de Links das páginas do MatrículaWeb PÓS-GRADUAÇÃO: https://matriculaweb.unb.br/graduacao/sec/qr.aspx
link_oferta_campus_pgr = "https://matriculaweb.unb.br/posgraduacao/oferta_dep.aspx?cod="
link_oferta_departamento_pgr = "https://matriculaweb.unb.br/posgraduacao/oferta_dis.aspx?cod="
link_oferta_materia_pgr = "https://matriculaweb.unb.br/posgraduacao/oferta_dados.aspx?cod="

# Listas com tipos de links (graduação e pós-graduação, respectivamente)
links_oferta_campus = [link_oferta_campus_gr, link_oferta_campus_pgr]
links_oferta_departamento = [link_oferta_departamento_gr, link_oferta_departamento_pgr]
links_oferta_materia = [link_oferta_materia_gr, link_oferta_materia_pgr]

'''modalidades = {'grad': {'campus': link_oferta_campus_gr, 'departamento' : link_oferta_departamento_gr, 'materia' : link_oferta_materia_gr}, 
               'pos' : {'campus': link_oferta_campus_pgr, 'departamento' : link_oferta_departamento_pgr, 'materia' : link_oferta_materia_pgr}}

for modalidade, links in modalidades.items():
  print(modalidade)
  for tipo, link in links.items():
    print(tipo, link)'''

# Listas para informações da extração
codigos_campus = [1,2,3,4]

# [0. código departamento, 1. sigla departamento, 2. nome departamento, 3. código campus]
departamentos = []

# [0. código materia, 1. nome matéria, 2. código departamento]
materias = []

# [0. nome professor,  1. nome matéria, 2. código matéria]
professores = []


# Variáveis para 'data'
now = datetime.now()
dias = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado", "Domingo"]
dia_semana = now.strftime("%A")                                                 # Dia da semana escrito
dia_ano = now.strftime("%j")                                                    # Número do dia do ano. (001-366)
numero_semana = now.strftime("%U")                                              # Número da semana do ano, com o primeiro dia sendo Domingo
mes = now.month
ano = now.year

# Extração do semestre atual do MW
link_qual_semestre = "https://matriculaweb.unb.br/graduacao/default.aspx"       
page = urllib.request.urlopen(link_qual_semestre)
soup = BeautifulSoup(page, 'lxml')
tabela_oferta = soup.find('span', attrs={'class': 'badge'})
semestre = tabela_oferta.find(text=True)[5:]


In [0]:
# Conectar no banco de dados
def conecta_sql():
    connection = pymysql.connect(host='192.185.222.70',
                                user='ruben703_master',
                                password='(@gmfsv[SP1f',
                                db='ruben703_avalia',
                                charset='utf8mb4',
                                cursorclass=pymysql.cursors.DictCursor)

    return pymysql.cursors.Cursor(connection)

cursor = conecta_sql()

def limpa_db():
    sql = "DROP TABLE `campi`, `departamentos`, `horarios_local_materias_004`, `horarios_local_materias_025`, `horarios_local_materias_039`, `horarios_local_materias_052`, `horarios_local_materias_12`, `horarios_local_materias_13`, `horarios_local_materias_14`, `horarios_local_materias_16`, `horarios_local_materias_17`, `horarios_local_materias_19`, `horarios_local_materias_40`, `horarios_local_materias_113`, `horarios_local_materias_115`, `horarios_local_materias_116`, `horarios_local_materias_120`, `horarios_local_materias_121`, `horarios_local_materias_125`, `horarios_local_materias_126`, `horarios_local_materias_127`, `horarios_local_materias_128`, `horarios_local_materias_129`, `horarios_local_materias_134`, `horarios_local_materias_135`, `horarios_local_materias_136`, `horarios_local_materias_137`, `horarios_local_materias_138`, `horarios_local_materias_139`, `horarios_local_materias_145`, `horarios_local_materias_147`, `horarios_local_materias_148`, `horarios_local_materias_149`, `horarios_local_materias_156`, `horarios_local_materias_157`, `horarios_local_materias_158`, `horarios_local_materias_159`, `horarios_local_materias_162`, `horarios_local_materias_163`, `horarios_local_materias_164`, `horarios_local_materias_165`, `horarios_local_materias_171`, `horarios_local_materias_176`, `horarios_local_materias_177`, `horarios_local_materias_178`, `horarios_local_materias_179`, `horarios_local_materias_191`, `horarios_local_materias_192`, `horarios_local_materias_194`, `horarios_local_materias_352`, `horarios_local_materias_372`, `horarios_local_materias_382`, `horarios_local_materias_383`, `horarios_local_materias_402`, `horarios_local_materias_404`, `horarios_local_materias_405`, `horarios_local_materias_406`, `horarios_local_materias_410`, `horarios_local_materias_411`, `horarios_local_materias_412`, `horarios_local_materias_413`, `horarios_local_materias_414`, `horarios_local_materias_420`, `horarios_local_materias_421`, `horarios_local_materias_422`, `horarios_local_materias_423`, `horarios_local_materias_424`, `horarios_local_materias_430`, `horarios_local_materias_431`, `horarios_local_materias_432`, `horarios_local_materias_440`, `horarios_local_materias_500`, `horarios_local_materias_530`, `horarios_local_materias_540`, `horarios_local_materias_550`, `horarios_local_materias_560`, `horarios_local_materias_570`, `horarios_local_materias_588`, `horarios_local_materias_589`, `horarios_local_materias_638`, `horarios_local_materias_650`, `horarios_local_materias_660`, `horarios_local_materias_700`, `horarios_local_materias_725`, `materias_004`, `materias_025`, `materias_039`, `materias_052`, `materias_12`, `materias_13`, `materias_14`, `materias_16`, `materias_17`, `materias_19`, `materias_40`, `materias_113`, `materias_115`, `materias_116`, `materias_120`, `materias_121`, `materias_125`, `materias_126`, `materias_127`, `materias_128`, `materias_129`, `materias_134`, `materias_135`, `materias_136`, `materias_137`, `materias_138`, `materias_139`, `materias_145`, `materias_147`, `materias_148`, `materias_149`, `materias_156`, `materias_157`, `materias_158`, `materias_159`, `materias_162`, `materias_163`, `materias_164`, `materias_165`, `materias_171`, `materias_176`, `materias_177`, `materias_178`, `materias_179`, `materias_191`, `materias_192`, `materias_194`, `materias_352`, `materias_372`, `materias_382`, `materias_383`, `materias_402`, `materias_404`, `materias_405`, `materias_406`, `materias_410`, `materias_411`, `materias_412`, `materias_413`, `materias_414`, `materias_420`, `materias_421`, `materias_422`, `materias_423`, `materias_424`, `materias_430`, `materias_431`, `materias_432`, `materias_440`, `materias_500`, `materias_530`, `materias_540`, `materias_550`, `materias_560`, `materias_570`, `materias_588`, `materias_589`, `materias_638`, `materias_650`, `materias_660`, `materias_700`, `materias_725`, `materias_detalhe_004`, `materias_detalhe_025`, `materias_detalhe_039`, `materias_detalhe_052`, `materias_detalhe_12`, `materias_detalhe_13`, `materias_detalhe_14`, `materias_detalhe_16`, `materias_detalhe_17`, `materias_detalhe_19`, `materias_detalhe_40`, `materias_detalhe_113`, `materias_detalhe_115`, `materias_detalhe_116`, `materias_detalhe_120`, `materias_detalhe_121`, `materias_detalhe_125`, `materias_detalhe_126`, `materias_detalhe_127`, `materias_detalhe_128`, `materias_detalhe_129`, `materias_detalhe_134`, `materias_detalhe_135`, `materias_detalhe_136`, `materias_detalhe_137`, `materias_detalhe_138`, `materias_detalhe_139`, `materias_detalhe_145`, `materias_detalhe_147`, `materias_detalhe_148`, `materias_detalhe_149`, `materias_detalhe_156`, `materias_detalhe_157`, `materias_detalhe_158`, `materias_detalhe_159`, `materias_detalhe_162`, `materias_detalhe_163`, `materias_detalhe_164`, `materias_detalhe_165`, `materias_detalhe_171`, `materias_detalhe_176`, `materias_detalhe_177`, `materias_detalhe_178`, `materias_detalhe_179`, `materias_detalhe_191`, `materias_detalhe_192`, `materias_detalhe_194`, `materias_detalhe_352`, `materias_detalhe_372`, `materias_detalhe_382`, `materias_detalhe_383`, `materias_detalhe_402`, `materias_detalhe_404`, `materias_detalhe_405`, `materias_detalhe_406`, `materias_detalhe_410`, `materias_detalhe_411`, `materias_detalhe_412`, `materias_detalhe_413`, `materias_detalhe_414`, `materias_detalhe_420`, `materias_detalhe_421`, `materias_detalhe_422`, `materias_detalhe_423`, `materias_detalhe_424`, `materias_detalhe_430`, `materias_detalhe_431`, `materias_detalhe_432`, `materias_detalhe_440`, `materias_detalhe_500`, `materias_detalhe_530`, `materias_detalhe_540`, `materias_detalhe_550`, `materias_detalhe_560`, `materias_detalhe_570`, `materias_detalhe_588`, `materias_detalhe_589`, `materias_detalhe_638`, `materias_detalhe_650`, `materias_detalhe_660`, `materias_detalhe_700`, `materias_detalhe_725`, `professores_004`, `professores_025`, `professores_039`, `professores_052`, `professores_12`, `professores_13`, `professores_14`, `professores_16`, `professores_17`, `professores_19`, `professores_40`, `professores_113`, `professores_115`, `professores_116`, `professores_120`, `professores_121`, `professores_125`, `professores_126`, `professores_127`, `professores_128`, `professores_129`, `professores_134`, `professores_135`, `professores_136`, `professores_137`, `professores_138`, `professores_139`, `professores_145`, `professores_147`, `professores_148`, `professores_149`, `professores_156`, `professores_157`, `professores_158`, `professores_159`, `professores_162`, `professores_163`, `professores_164`, `professores_165`, `professores_171`, `professores_176`, `professores_177`, `professores_178`, `professores_179`, `professores_191`, `professores_192`, `professores_194`, `professores_352`, `professores_372`, `professores_382`, `professores_383`, `professores_402`, `professores_404`, `professores_405`, `professores_406`, `professores_410`, `professores_411`, `professores_412`, `professores_413`, `professores_414`, `professores_420`, `professores_421`, `professores_422`, `professores_423`, `professores_424`, `professores_430`, `professores_431`, `professores_432`, `professores_440`, `professores_500`, `professores_530`, `professores_540`, `professores_550`, `professores_560`, `professores_570`, `professores_588`, `professores_589`, `professores_638`, `professores_650`, `professores_660`, `professores_700`, `professores_725`, `reservas_004`, `reservas_025`, `reservas_039`, `reservas_052`, `reservas_12`, `reservas_13`, `reservas_14`, `reservas_16`, `reservas_17`, `reservas_19`, `reservas_40`, `reservas_113`, `reservas_115`, `reservas_116`, `reservas_120`, `reservas_121`, `reservas_125`, `reservas_126`, `reservas_127`, `reservas_128`, `reservas_129`, `reservas_134`, `reservas_135`, `reservas_136`, `reservas_137`, `reservas_138`, `reservas_139`, `reservas_145`, `reservas_147`, `reservas_148`, `reservas_149`, `reservas_156`, `reservas_157`, `reservas_158`, `reservas_159`, `reservas_162`, `reservas_163`, `reservas_164`, `reservas_165`, `reservas_171`, `reservas_176`, `reservas_177`, `reservas_178`, `reservas_179`, `reservas_191`, `reservas_192`, `reservas_194`, `reservas_352`, `reservas_372`, `reservas_382`, `reservas_383`, `reservas_402`, `reservas_404`, `reservas_405`, `reservas_406`, `reservas_410`, `reservas_411`, `reservas_412`, `reservas_413`, `reservas_414`, `reservas_420`, `reservas_421`, `reservas_422`, `reservas_423`, `reservas_424`, `reservas_430`, `reservas_431`, `reservas_432`, `reservas_440`, `reservas_500`, `reservas_530`, `reservas_540`, `reservas_550`, `reservas_560`, `reservas_570`, `reservas_588`, `reservas_589`, `reservas_638`, `reservas_650`, `reservas_660`, `reservas_700`, `reservas_725`;"
    try: cursor.execute(sql)
    except pymysql.Error as e: print("Erro ao limpar db. %d: %s" % (e.args[0], e.args[1]))

def limpa_professores():
  cursor.execute()


In [0]:
# Entrar na página de cada 'Campus' escolhido e extrair os 'Departamentos' que possuem oferta cadastrada
for x in [0,1]:                                                                 # pega links da gr e da pgr
    if x==0: titulo = 'graduacao'
    else: titulo = 'pos'

    for codigo in tqdm_notebook(codigos_campus, desc='Departamentos_campus_'+titulo+' (python)'):
        link_campus = links_oferta_campus[x]+str(codigo)
        page = urllib.request.urlopen(link_campus)
        soup = BeautifulSoup(page, 'lxml')
        tabela_oferta = soup.findAll('tr')

        for linha in tabela_oferta:
            departamento = []
            cells = linha.findAll('td')
            if len(cells)==3:                                                   # elimina busca da página, que também possui formato de tabela
                departamento.append(cells[0].get_text())                        # código departamento
                departamento.append(cells[1].get_text())                        # sigla departamento
                departamento.append(cells[2].get_text())                        # nome departamento
                departamento.append(str(codigo))                                # código campus
                departamento.append(titulo)                                     # modalidade
                departamentos.append(departamento)


In [0]:
# Cria tabela com campus e tabela para departamentos
try: cursor.execute("CREATE TABLE `campi` (`cod` int(11) NOT NULL PRIMARY KEY, `nome` varchar(255) COLLATE utf8_unicode_ci NOT NULL) ENGINE=MyISAM DEFAULT CHARSET=utf8 COLLATE=utf8_unicode_ci;")
except pymysql.Error as e: print("Error %d: %s" % (e.args[0], e.args[1]))

try: cursor.execute("INSERT INTO `campi` (`cod`, `nome`) VALUES (1, 'Darcy Ribeiro'), (2, 'Planaltina'), (3, 'Ceilândia'), (4, 'Gama');")
except pymysql.Error as e: print("Error %d: %s" % (e.args[0], e.args[1]))

try: cursor.execute("CREATE TABLE `departamentos` (`cod` int(11) NOT NULL PRIMARY KEY, sigla varchar(4), `nome` varchar(255), `cod_campus` int(11), `modalidade` varchar(10) COLLATE utf8_unicode_ci NOT NULL) ENGINE=MyISAM DEFAULT CHARSET=utf8 COLLATE=utf8_unicode_ci;")
except pymysql.Error as e: print("Error %d: %s" % (e.args[0], e.args[1]))


# Cria tabela para créditos
try: cursor.execute("CREATE TABLE `creditos` (`chave` varchar(20) NOT NULL PRIMARY KEY, `teoricos` varchar(4), `praticos` varchar(4), `extensao` varchar(4), `estudos` varchar(4) COLLATE utf8_unicode_ci NOT NULL) ENGINE=MyISAM DEFAULT CHARSET=utf8 COLLATE=utf8_unicode_ci;")
except pymysql.Error as e: print("Error %d: %s" % (e.args[0], e.args[1]))


In [0]:
# Inserir departamentos extraídos no banco de dados
for departamento in tqdm_notebook(departamentos, desc='Departamentos (sql)'):
    if sum(x.count(departamento[0]) for x in departamentos) == 2: departamento[4] = 'ambos'
    
    # [0. código departamento, 1. sigla departamento, 2. nome departamento, 3. código campus, 4. modalidade]
    try:  cursor.execute("INSERT INTO departamentos (cod, sigla, nome, cod_campus, modalidade) VALUES ('{}','{}','{}','{}','{}');".format(departamento[0], departamento[1], departamento[2], departamento[3], departamento[4]))
    except pymysql.Error as e:  print("Error %d: %s" % (e.args[0], e.args[1]))


In [0]:
# Entrar na página de cada 'Departamento' do campus escolhido e extrair as 'Matérias' que possuem oferta cadastrada
for x in [0, 1]:                                                                # pega links da gr e da pgr
    if x==0: titulo = 'graduacao'
    else: titulo = 'pos'

    for departamento in tqdm_notebook(departamentos, desc='Matérias_departamento_'+titulo+ '(python)'):
        link_departamento = links_oferta_departamento[x]+departamento[0]
        page = urllib.request.urlopen(link_departamento)
        soup = BeautifulSoup(page, 'lxml')
        tabela_departamento = soup.findAll('tr')
        
        if tabela_departamento != None:
            cursor.execute("CREATE TABLE IF NOT EXISTS materias_{} (cod INT PRIMARY KEY, nome VARCHAR(255), cod_departamento INT(11) NOT NULL, modalidade varchar(10));".format(departamento[0]))              #Cria uma tabela de matérias para cada departamento
            for linha in tabela_departamento:
                materia = []
                cells = linha.findAll('td')
                if len(cells)==3:
                    materia.append(cells[0].get_text())
                    materia.append(cells[1].get_text())
                    materia.append(departamento[0])
                    materia.append(titulo)
                    materias.append(materia)


In [0]:
# Inserir matérias extraídas no banco de dados
for materia in tqdm_notebook(materias, desc='Matérias (sql)'):
    # [0. código materia, 1. nome materia, 2. código departamento]
    try: cursor.execute("INSERT INTO materias_{} (cod, nome, cod_departamento, modalidade) VALUES ('{}', '{}', '{}', '{}');".format(materia[2], materia[0], materia[1], materia[2], materia[3]))
    except pymysql.Error as e: print("Error %d: %s" % (e.args[0], e.args[1]))


In [0]:
# Cria tabelas materias_detalhe_{COD_DEPARTAMENTO} para todos os departamentos
for departamento in tqdm_notebook(departamentos, desc='MatériasDetalhe_departamento cria tabelas (sql)'):
    # A chave será composta de COD_DEPARTAMENTO - COD_MATERIA - TURMA
    try: cursor.execute("CREATE TABLE IF NOT EXISTS materias_detalhe_{} (chave VARCHAR(25) PRIMARY KEY, cod_materia INT(15) NOT NULL, cod_departamento INT(15) NOT NULL, turma VARCHAR(5), semestre VARCHAR(1), ano INT(4), vagas_total INT(3), vagas_ocupadas INT(3), vagas_restantes INT(3), turno VARCHAR(30), nome_turma VARCHAR(255));".format(departamento[0]))
    except pymysql.Error as e: print("Error %d: %s" % (e.args[0], e.args[1]))

    try: cursor.execute("CREATE TABLE IF NOT EXISTS horarios_local_materias_{} (chave VARCHAR(25), dia VARCHAR(90), hora_inicio VARCHAR(10), hora_fim VARCHAR(10), local VARCHAR(255), semestre VARCHAR(1), ano INT(4), PRIMARY KEY(chave, dia, hora_inicio));".format(departamento[0]))
    except pymysql.Error as e: print("Error %d: %s" % (e.args[0], e.args[1]))

    try: cursor.execute("CREATE TABLE IF NOT EXISTS professores_{} (chave VARCHAR(25), nome VARCHAR(90), cod_departamento INT(15) NOT NULL, semestre VARCHAR(1), ano INT(4), PRIMARY KEY(chave, nome));".format(departamento[0]))
    except pymysql.Error as e: print("Error %d: %s" % (e.args[0], e.args[1]))

    try: cursor.execute("CREATE TABLE IF NOT EXISTS reservas_{} (chave VARCHAR(25), curso VARCHAR(90), vagas_total INT(10), vagas_calouros INT(10), PRIMARY KEY(chave, curso));".format(departamento[0]))
    except pymysql.Error as e: print("Error %d: %s" % (e.args[0], e.args[1]))
    

In [0]:
# Entrar na página de cada 'Matéria' do campus escolhido e extrair os 'detalhes' de cada uma
# Frases que indicam na lista da extração que os 'professores' acabaram
criterios_parada_professor = ["Reserva para curso", "Turma"]

numero_materia_adicionada = 0
#materiasc = [['206458', 'SEMINÁRIOS EM TURISMO 1', '039', 'graduacao'], ['112020', 'CRISTALOGRAFIA', '440', 'graduacao'], ['207438', 'ESTÁGIO SUPERVISIONADO EM ENGENHARIA DE COMPUTAÇÃO 1', '163', 'graduacao'], ['142573', 'INGLÊS INSTRUMENTAL 2', '149', 'graduacao'], ['170691', 'BROMATOLOGIA', '171', 'graduacao']]

for materia in tqdm_notebook(materias, desc='Matérias Detalhe'):
    if materia[3] == 'graduacao': x=0                                           # 'x' para dizer se o link acessado será da graduação ou da pós
    else: x=1

    link_materia = links_oferta_materia[x]+materia[0]+"&dep="+materia[2]
    page = urllib.request.urlopen(link_materia)
    soup = BeautifulSoup(page, 'lxml')

    # Atributo de classe abaixo filtra exatamente as 'caixas' de cada turma em uma página de matéria no MW
    tabela_turmas = soup.findAll('table', attrs={'id': 'datatable', 'class': 'table table-striped table-bordered tabela-oferta'})
    tabela_creditos = (soup.findAll('div', attrs={'class': 'table-responsive'}))[0]
    
    for creditos in tabela_creditos:
        informacoes = re.search("(\d+)-(\d+)-(\d+)-(\d+)", str(creditos))
        if informacoes == None:
            continue
        else:
            creditos = informacoes.groups()
            Teor = creditos[0]
            Prat = creditos[1]
            Ext = creditos[2]
            Est = creditos[3]
    
    chave_creditos = str(materia[2]) + "-" + str(materia[0])
    try: cursor.execute("INSERT INTO creditos (chave, teoricos, praticos, extensao, estudos) VALUES ('{}','{}','{}','{}','{}');".format(chave_creditos, Teor, Prat, Ext, Est))
    except pymysql.Error as e: print("Error %d: %s" % (e.args[0], e.args[1]))

    #print(numero_materia_adicionada, "Matéria:", materia)
    #print(link_materia)

    for linha in tabela_turmas:
        extracao = (linha.findAll(text=True))[6:]
        #print(extracao)

        # [0. turma, 1. vagas total, 2. vagas ocupadas, 3. vagas restantes, 4. turno, 5. horarios (lista de listas), 6. professores (lista), 7. reserva (lista de listas), 8. nome da turma]
        turma = []

        # [[0. dia, 1. início aula, 2. fim aula, 3. local], n]
        horarios = []

        # [(n) nome]
        professores = []

        turma_nome = ''
        possui_executor = ''

        x = 0
        turma.append(extracao[x])                                               # Letra da turma
        if((extracao[x+1])[:9] == "Executor:"):
            x = x + 1
            possui_executor = True
        turma.append(extracao[x+3])                                             # Total de vagas
        turma.append(extracao[x+5])                                             # Vagas ocupadas
        turma.append(extracao[x+7])                                             # Vagas restantes

        if (possui_executor): x = 9
        else: x = 8

        if (extracao[x] == "Calouros"):                                         
            x = x + 7                                                           # A informação não é extraída agora porque ela se repete mais pro final da lista de extração
            turma.append(extracao[x])                                           # Turno
        else:
            turma.append(extracao[x])                                           # Turno

        x = x + 1
        while extracao[x] in dias:                                              # As informações sobre horários das aulas vêm sequencialmente na extração, 
            # [0. dia, 1. hora início, 2. hora fim, 3. Local]                   # loop para identificar o fim das infos
            infos = []                                                          
            infos.append(extracao[x])                                           # Dia
            infos.append(extracao[x+1])                                         # Hora início
            infos.append(extracao[x+2])                                         # Hora fim
            infos.append(extracao[x+3])                                         # Local
            horarios.append(infos)
            x = x + 4
        turma.append(horarios)

        while extracao[x] not in criterios_parada_professor:                    # Professores que dão aula para uma mesma turma também vem de forma sequencial
            professores.append(extracao[x])                                     # Se não for um critério de parada, ir salvando todos os nomes
            if len(extracao) > (x+1): x = x + 1
            else: break
        turma.append(professores)
        
        try: 
            x = extracao.index("Reserva para curso")
            pos_lista = x
        except: 
            pos_lista = 0
        # [0. curso, 1. vagas reservadas, 2. vagas reservadas especialmente para calouros]
        reservas = []
        if (extracao[x] == "Reserva para curso"):
            while (pos_lista != len(extracao)):
                if x+4 > len(extracao): break
                r = []
                r.append(extracao[x+3])                                         # Curso da rerva
                r.append(extracao[x+4])                                         # Vagas reservadas
                r.append(extracao[x+5])                                         # Vagas reservadas para calouros
                reservas.append(r)
                pos_lista = extracao.index(extracao[x+3])+3
                x = x + 3

        turma.append(reservas)

        try: x = extracao.index("Turma")
        except: a = 0
        if extracao[x] == "Turma":                                              
            turma_nome = extracao[x+1]
        turma.append(turma_nome)

        #print(turma)


        # A chave será composta de COD_DEPARTAMENTO - COD_MATERIA - TURMA
        chave = str(materia[2]) + "-" + str(materia[0]) + "-" + str(turma[0])

        #print("INSERT INTO materias_detalhe_{} (chave, cod_materia, cod_departamento, turma, semestre, ano, vagas_total, vagas_ocupadas, vagas_restantes, turno, nome_turma) VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}');".format(materia[2], chave, materia[0], materia[2], turma[0], semestre[:2], ano, int(turma[1]), int(turma[2]), int(turma[3]), turma[4], turma_nome))
        try: cursor.execute("INSERT INTO materias_detalhe_{} (chave, cod_materia, cod_departamento, turma, semestre, ano, vagas_total, vagas_ocupadas, vagas_restantes, turno, nome_turma) VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}');".format(materia[2], chave, materia[0], materia[2], turma[0], semestre[:2], ano, int(turma[1]), int(turma[2]), int(turma[3]), turma[4], turma_nome))
        except pymysql.Error as e: print("Error materias_detalhe %d: %s" % (e.args[0], e.args[1]))

        for horario in horarios:
            #print("INSERT INTO horarios_local_materias_{} (chave, dia, hora_inicio, hora_fim, local, semestre, ano) VALUES ('{}','{}','{}','{}','{}','{}','{}');".format(materia[2], chave, horario[0], horario[1], horario[2], horario[3], semestre[:2], ano))
            try: cursor.execute("INSERT INTO horarios_local_materias_{} (chave, dia, hora_inicio, hora_fim, local, semestre, ano) VALUES ('{}','{}','{}','{}','{}','{}','{}');".format(materia[2], chave, horario[0], horario[1], horario[2], horario[3], semestre[:2], ano))
            except pymysql.Error as e: print("Error horarios_local_materias %d: %s" % (e.args[0], e.args[1]))

        for professor in professores:
            #print("INSERT INTO professores_{} (chave, nome, semestre, ano) VALUES ('{}','{}','{}','{}');".format(materia[2], chave, professor, semestre[:2], ano))
            try: cursor.execute("INSERT INTO professores_{} (chave, nome, cod_departamento, semestre, ano) VALUES ('{}','{}','{}','{}');".format(materia[2], chave, professor, materia[2], semestre[:2], ano))
            except pymysql.Error as e: print("Error professores %d: %s" % (e.args[0], e.args[1]))

        for reserva in reservas:
            #print("INSERT INTO reservas_{} (chave, curso, vagas_total, vagas_calouros) VALUES ('{}','{}','{}','{}');".format(materia[2], chave, reserva[0], reserva[1], reserva[2]))
            try: cursor.execute("INSERT INTO reservas_{} (chave, curso, vagas_total, vagas_calouros) VALUES ('{}','{}','{}','{}');".format(materia[2], chave, reserva[0], reserva[1], reserva[2]))
            except pymysql.Error as e: print("Error reserva %d: %s" % (e.args[0], e.args[1]))

    #print("\n")
    numero_materia_adicionada = numero_materia_adicionada + 1
